In [39]:
import os
import openai  # Ben bu çalışmada, openai kütüphanesinin "openai==0.27.6" sürümünü kullandım.
from dotenv import load_dotenv
import warnings
from requests.packages.urllib3.exceptions import NotOpenSSLWarning

warnings.simplefilter('ignore', NotOpenSSLWarning)
load_dotenv()

# Daha önce azure üzerinde deploy ettiğim gpt-3.5 Turbo modelini kullanacağım.
openai.api_type = "azure"
openai.api_base = os.environ.get("OPENAI_API_BASE")
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_version = "2023-05-15"

DEPLOYMENT_NAME =  os.environ.get("OPENAI_CHAT_DEPLOYMENT_NAME") 

In [8]:
def chat(prompt, model=DEPLOYMENT_NAME):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine= DEPLOYMENT_NAME,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

### Sentiment Analysis

In [31]:

sampletext = f"""Yakın zamanda SGE Teknoloji, TKK LED Işık Ampulünü satın aldım ve performansından tamamen büyülendim diyebilirim. Bu sıradan bir LED ampul değil, gerçekten bir oyun değiştirici!

Öncelikle, bu ampulün ürettiği parlaklık seviyesi eşsiz. İster okuma odanız için kullanın ister oturma odasında tavan ışığı olarak, mekanı etkileyici, net ve temiz bir ışıkla aydınlatıyor. Gün ışığının yakalanıp bir ampule konduğunu hissettiriyor! Ayarlanabilir parlaklık özelliği de oldukça kullanışlı, ruh halinize göre ortamı ayarlamanıza olanak tanıyor.

SGE Teknoloji, TKK LED Işık Ampulünün enerji verimliliği özellikle bahsedilmeyi hak ediyor. Yüksek parlaklığına rağmen, geleneksel ampullere kıyasla önemli ölçüde daha az enerji tüketiyor. Enerji maliyetlerini düşürmeyi hedefleyen herkes için büyük bir artı. Elektrik faturamızda bile fark edilir bir azalma oldu, bu ampul kesinlikle değerli bir yatırım.

Ömür açısından, TKK LED diğer kullandığım tüm ampullerden daha iyi performans gösteriyor. Üç ay oldu ve hala ilk takıldığı günkü kadar parlak. Üretici 25,000 saat ömür vaat ediyor ve şimdiye kadar gösterdiği performansı göz önünde bulundurursak, bu iddiayı sorgulamak için hiçbir sebep yok.

Ampulün tasarımı şık ve modern, evimizin çağdaş dekorasyonuyla iyi uyum sağlıyor. Kurulumu kolay ve Koç'tan bir ürünle ilgili bir sorum olduğunda müşteri hizmetleri mükemmeldi.

Sonuç olarak, TKK LED Işık Ampulü, her açıdan beklentilerimi aştı. Parlak, enerji verimli, uzun ömürlü ve iyi tasarlanmış. Sadece bir ışık ampulü değil, her ev için akıllıca bir yatırım. Üstün bir aydınlatma çözümü arayan herkese bu ürünü şiddetle tavsiye ederim."""

prompt = f"""
Aşağıda kullanıcı tarafından yapılan ürün yorumunun duygusu nedir, sentiment analaysis yap.
iki çizgi arasına yerleştirilmiş olan? Metini incele.
Bana cevabını bir kelimeyle ver. (Positive, Negative ve benzeri)
---
{sampletext}
---
"""
response = chat(prompt)
print(response)

Positive


### Entity and Keyword Detection

In [32]:
prompt = f"""
Aşağıda kullanıcı tarafından yapılan yorumdan belirtilen öğeleneri çıkar.
- İnceleyen kişinin satın aldığı ürün
- Ürünü üreten şirket

Listeyi bir JSON dizisinde sağlayın.

---
{sampletext}
---
"""
response = chat(prompt)
print(response)

{
    "ürün": "TKK LED Işık Ampulü",
    "üretici": "SGE Teknoloji"
}


### Inferring Specifications

In [35]:
prompt = f"""
Incelenen metinde, çizgilerle ayrılan bölümden aşağıdaki unsurları belirleyin:

- Üründe olumlu (positive sentiment) izlenim yaratan özellikler (features)
- Üründe olumsuz (negative sentiment) izlenim yaratan özellikler (features)

Tüm özellik ve yönleri, bir ürün spesifikasyonu şeklinde yeniden ifade edin.

Her bir duygu altında spesifikasyonların (Özelliklerin değil) listesini sunun.
---
{sampletext}
---
"""
response = chat(prompt)
print(response)

Olumlu Özellikler:
- Eşsiz parlaklık seviyesi
- Net ve temiz ışık
- Ayarlanabilir parlaklık özelliği
- Yüksek enerji verimliliği
- Uzun ömürlü
- Şık ve modern tasarım
- Kolay kurulum
- Mükemmel müşteri hizmetleri

Olumsuz Özellikler: N/A

Ürün Spesifikasyonu:
- Ampul tipi: LED
- Parlaklık seviyesi: Yüksek
- Ayarlanabilir parlaklık: Var
- Enerji verimliliği: Yüksek
- Ömür: 25,000 saat
- Tasarım: Şık ve modern
- Kurulum: Kolay
- Müşteri hizmetleri: Mükemmel


### Intent Detection

In [49]:
sample_text_tr = f"""Yarın saat 08:00 için İstanbul havaalanından Paris'e bilet alabilir miyim?"""

prompt = f"""
Aşağıda listelenen eylemleri göz önünde bulundurun
- Uçuş rezervasyonu yapmak
- Araba kiralamak
- Otobüs bileti satın almak

İki çizgi arasında verilen aşağıdaki metne dayanarak
- kullanıcının hangi eylemi gerçekleştirmeye çalıştığı (action)
- metinden eylemle ilgili olan anahtar kelimeleri çıkarın (keywords)
- bulunursa eylemin hedefini belirtin (destination)
- bulunursa eylem için bir zaman belirtin (time)
- bulunursa eylem için bir tarih belirtin. Bugün 8 Eylül 2023. (date)
- bulunursa eylemin kaynağını belirtin (nerden nereye) (source)

Yanıtınızı bir JSON nesnesi olarak verin.
---
{sample_text_tr}
---
"""

response = chat(prompt)
print(response)

{
  "action": "Uçuş rezervasyonu yapmak",
  "keywords": ["bilet", "İstanbul havaalanı", "Paris", "yarin", "08:00"],
  "destination": "Paris",
  "time": "08:00",
  "date": "9 Eylül 2023",
  "source": "İstanbul havaalanı"
}
